In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('Spotify_song.csv')
df.head()

,Unnamed: 0,Song,Album,Artist Names,Artist(s) Genres,Hot100 Ranking Year,Hot100 Rank,Song Length(ms),Popularity,Acousticness,Danceability,Energy,Instrumentalness,Liveness,Loudness,Speechiness,Tempo,Valence
0,0,Prisoner of Love - Remastered - 1992,Today & Yesterday,Perry Como,"adult standards, easy listening",1946,1,211866,19,0.767,0.247,0.1820,0.00209,0.1670,-11.121,0.0328,80.064,0.185
1,1,To Each His Own,The Best Of The Mercury Years,Eddy Howard,british dance band,1946,2,184293,25,0.947,0.344,0.0596,0.00006,0.1810,-16.766,0.0394,81.037,0.150
2,2,The Gypsy,The Anthology,The Ink Spots,"vocal harmony group, lounge",1946,3,164533,33,0.812,0.531,0.1250,0.00000,0.1030,-15.463,0.0552,76.056,0.222
3,3,Five Minutes More,The Columbia Years (1943-1952): The Complete R...,Frank Sinatra,"adult standards, lounge, easy listening",1946,4,154773,31,0.794,0.670,0.0625,0.00000,0.0762,-20.393,0.0611,142.894,0.569
4,4,Rumors Are Flying,1946 Broadcasts,Frankie Carle,"space age pop, honky-tonk piano",1946,5,183133,27,0.951,0.332,0.2040,0.00252,0.6380,-15.557,0.0437,72.355,0.377


In [3]:
df = df.drop(['Unnamed: 0','Hot100 Ranking Year','Song Length(ms)','Hot100 Ranking Year','Hot100 Rank','Song Length(ms)','Popularity','Acousticness','Danceability','Energy','Instrumentalness','Liveness','Loudness','Speechiness','Tempo','Valence'],axis=1)

In [4]:
df.isnull().sum()

Song                0
Album               0
Artist Names        0
Artist(s) Genres    0
dtype: int64

In [5]:
df.columns

Index(['Song', 'Album', 'Artist Names', 'Artist(s) Genres'], dtype='object')

In [6]:
df.shape

(6879, 4)

In [7]:
df = df.rename(columns={
    'Artist Names': 'Artists',
    'Artist(s) Genres': 'Genres',
})

In [8]:
df.head(1)

,Song,Album,Artists,Genres
0,Prisoner of Love - Remastered - 1992,Today & Yesterday,Perry Como,"adult standards, easy listening"


In [9]:
df.iloc[6000].Genres

' broadway, hollywood, show tunes, movie tunes '

In [10]:
df.iloc[6000].Album

'Frozen (Original Motion Picture Soundtrack / Deluxe Edition)'

In [11]:
import re
df['Album'] = df['Album'].apply(lambda x: re.sub(r"\(.*?\)/", "", x).strip())

In [12]:
df['Song'] = df['Song'].apply(lambda x: re.sub(r"\(.*?\)", "", x).strip())

In [13]:
df['Artists'] = df['Artists'].str.replace(" ", "")
df['Album'] = df['Album'].str.replace(" ", "")
df['Genres'] = df['Genres'].str.replace(" ", "")

In [14]:
df['Genres'][0]

'adultstandards,easylistening'

In [15]:
df['Artists'] = df['Artists'].apply(lambda x: x.split())
df['Genres'] = df['Genres'].apply(lambda x: x.split())
df['Album'] = df['Album'].apply(lambda x: x.split())

In [16]:
df['Tags'] = df['Album'] + df['Artists'] + df['Genres']

In [17]:
df = df.drop(['Album','Artists','Genres'],axis=1)

In [18]:
df['Tags'] = df['Tags'].apply(lambda x:" ".join(x))

In [19]:
df.head()

,Song,Tags
0,Prisoner of Love - Remastered - 1992,"Today&Yesterday PerryComo adultstandards,easyl..."
1,To Each His Own,TheBestOfTheMercuryYears EddyHoward britishdan...
2,The Gypsy,"TheAnthology TheInkSpots vocalharmonygroup,lounge"
3,Five Minutes More,TheColumbiaYears(1943-1952):TheCompleteRecordi...
4,Rumors Are Flying,"1946Broadcasts FrankieCarle spaceagepop,honky-..."


In [20]:
df['Tags'] = df['Tags'].apply(lambda x: x.lower())

In [21]:
import nltk
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [22]:
def stem(text):
    y = []
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)

In [23]:
df['Tags'] = df['Tags'].apply(stem)

In [24]:
df['Tags'][0]

'today&yesterday perrycomo adultstandards,easylisten'

In [25]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000,stop_words='english')

In [26]:
vectors = cv.fit_transform(df['Tags']).toarray()

In [27]:
vectors[0]

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [28]:
df['Song'].count()

6879

In [29]:
from sklearn.metrics.pairwise import cosine_similarity

In [30]:
similarity = cosine_similarity(vectors)

In [31]:
similarity[1]

array([0., 1., 0., ..., 0., 0., 0.])

In [32]:
sorted(list(enumerate(similarity[0])),reverse=True,key=lambda x: x[1])[1:11]

[(27, 0.9999999999999999),
 (396, 0.9999999999999999),
 (36, 0.7745966692414834),
 (213, 0.7745966692414834),
 (10, 0.6708203932499369),
 (54, 0.6708203932499369),
 (59, 0.6708203932499369),
 (112, 0.6708203932499369),
 (130, 0.6708203932499369),
 (141, 0.6708203932499369)]

In [99]:
def recommend(song):
    movie_index = df[df['Song'] == song].index[0]
    distances = similarity[movie_index]
    song_list = sorted(list(enumerate(distances)),reverse=True,key=lambda x: x[1])[1:11]

    
    for i in song_list:
        # print(df.iloc[i[0]].Song)
        print(i)

In [ ]:
C:\Users\DELL\OneDrive\Documents\Web Dev\Song_Recommender\music logo.png

In [103]:
recommend('Despacito')

(6471, 0.8451542547285165)
(5257, 0.7715167498104596)
(6778, 0.7559289460184544)
(6175, 0.6299407883487119)
(6459, 0.5976143046671968)
(6543, 0.5714285714285713)
(6761, 0.5669467095138407)
(1437, 0.5345224838248487)
(6468, 0.5202659817144719)
(6800, 0.50709255283711)


In [35]:
df.head()

,Song,Tags
0,Prisoner of Love - Remastered - 1992,"today&yesterday perrycomo adultstandards,easyl..."
1,To Each His Own,thebestofthemercuryyear eddyhoward britishdanc...
2,The Gypsy,"theantholog theinkspot vocalharmonygroup,loung"
3,Five Minutes More,thecolumbiayears(1943-1952):thecompleterecordi...
4,Rumors Are Flying,"1946broadcast frankiecarl spaceagepop,honky-to..."


In [36]:
df.to_csv('song_tags.csv')

In [37]:
import pickle

In [38]:
# pickle.dump(df,open('songs.pkl','wb'))

In [39]:
# pickle.dump(similarity,open('similarity.pkl','wb'))

In [40]:
df['Song'].values

array(['Prisoner of Love - Remastered - 1992', 'To Each His Own',
       'The Gypsy', ..., 'Unholy', 'One Mississippi',
       'Circles Around This Town'], dtype=object)

In [81]:
# pickle.dump(df.to_dict(),open('Song_dict.pkl','wb'))